## Parcels intersected with NRCS 2007 and Impervious
### Recreate these tables: 
#### F:\Research and Analysis\Reporting\Annual Reports\2016 Annual Report\Data and Analysis\Coverage\RawData\AnalysisProduct

#### F:/Research and Analysis/Reporting/Annual Reports/2016 Annual Report/Data and Analysis/Coverage/RawData


In [1]:
# import modules and set environment settings
import arcpy, os
arcpy.overwriteOutput = True

# set local variables
sdeBase = "F:/GIS/GIS_DATA/Vector.sde"
parcels = sdeBase + "//sde.SDE.Parcels//sde.SDE.Parcel_Master"
impervious = sdeBase + "//sde.SDE.Tahoe_Impervious_2010//sde.SDE.Impervious_2010"
soils = sdeBase + "//sde.SDE.Soils//sde.SDE.land_capability_NRCS2007" 
localplan = sdeBase + "//sde.SDE.Planning//sde.SDE.LocalPlan"

# delete in memory feature classes
if arcpy.Exists("in_memory/id_NRCS_LocalPlan"):
    arcpy.Delete_management("in_memory/id_NRCS_LocalPlan")

if arcpy.Exists("in_memory/id_NRCS_LocalPlan_Impervious"):
    arcpy.Delete_management("in_memory/id_NRCS_LocalPlan_Impervious")
    
if arcpy.Exists("in_memory/id_NRCS_LocalPlan_Impervious_Parcel"):
    arcpy.Delete_management("in_memory/id_NRCS_LocalPlan_Impervious_Parcel")

# tables
outTableImpervious = "id_NRCS_LocalPlan_Impervious.csv"
outTableParcel = "id_NRCS_LocalPlan_Impervious_Parcel.csv"
outLocation = r"F:/Research and Analysis/Reporting/Annual Reports/2017 Annual Report/2017-Annual-Report_Data/Coverage/rawdata"

# output feature classes
id_NRCS_LocalPlan = "in_memory/id_NRCS_LocalPlan"
id_NRCS_LocalPlan_Impervious = "in_memory/id_NRCS_LocalPlan_Impervious"
id_NRCS_LocalPlan_Impervious_Parcel = "in_memory/id_NRCS_LocalPlan_Impervious_Parcel"

####
####

# identify land capability/local plan 
arcpy.Identity_analysis (soils, localplan, id_NRCS_LocalPlan)

# identify land capability/local plan/impervious
arcpy.Identity_analysis (id_NRCS_LocalPlan, impervious, id_NRCS_LocalPlan_Impervious)

# calculate acres
with arcpy.da.UpdateCursor(id_NRCS_LocalPlan_Impervious, ['GISAcre', 'SHAPE@']) as cursor:
    for row in cursor:
        row[0] = row[1].getArea('GEODESIC', 'ACRES')
        cursor.updateRow(row)

# delete table if it exists
if arcpy.Exists(os.path.join(outLocation,outTableImpervious)):
    arcpy.Delete_management(os.path.join(outLocation,outTableImpervious))
    
# Execute Table To Table
arcpy.TableToTable_conversion(id_NRCS_LocalPlan_Impervious, outLocation, outTableImpervious)

# success?
print("Table {0} created".format(outTableImpervious))

####
####

# identify land capability/local plan/impervious/parcel master
arcpy.Identity_analysis (id_NRCS_LocalPlan_Impervious, parcels, id_NRCS_LocalPlan_Impervious_Parcel)

# calculate acres
with arcpy.da.UpdateCursor(id_NRCS_LocalPlan_Impervious_Parcel, ['GISAcre', 'SHAPE@']) as cursor:
    for row in cursor:
        row[0] = row[1].getArea('GEODESIC', 'ACRES')
        cursor.updateRow(row)

# delete table if it exists
if arcpy.Exists(os.path.join(outLocation,outTableParcel)):
    arcpy.Delete_management(os.path.join(outLocation,outTableParcel))

# Execute Table To Table
arcpy.TableToTable_conversion(id_NRCS_LocalPlan_Impervious_Parcel, outLocation, outTableParcel)

# success?
print("Table {0} created".format(outTableParcel))

Table id_NRCS_LocalPlan_Impervious.csv created
Table id_NRCS_LocalPlan_Impervious_Parcel.csv created
